## A)
Consider the following stochastic field: $E(x)=10(1+f(x))$
where $f(x)$ is a zero-mean stationary Gaussian field with unit variance and $x \in [0,5] (m)$.
The autocorrelation function for $f$ is $R_f (\tau) = \exp⁡(-|\tau|/2)$.

1. Use the Karhunen-Loeve series expansion method to generate $N = 5000$ realizations of the field $E(x)$.
2. Justify the number of terms you retained in the KL-expansion.
3. Calculate the ensemble average and the ensemble variance from these realizations. To which values would they converge as we increase the number $N$ of realizations?

---

***Answer***

Throughout this document we will refer to *Stochastic Finite Element Methods by Vissarion Papadopoulos & Dimitris G. Giovanis* as ***the book***.

We focus on the Gaussian field $f(x)$. To perform analysis with it, we center its domain in $[-2.5, 2.5]$ and we have $a = 2.5$.

Our covariance function is of the form defined in (2.22) of the book. Solving for its eigenvalues $\lambda_n$ and eigenfunctions $\varphi_n$ we get (book 2.24 - 2.28) 

* For n = odd,
    $$
    \lambda_{n}=\frac{2 b}{1+\omega_{n}^{2} b^{2}}, \quad \varphi_{n}(x)=c_{n} \cos \left(\omega_{n} x\right)
    $$
    where $c_n$ is given by 
    $$
    c_{n}=\frac{1}{\sqrt{a+\frac{\sin \left(2 \omega_{n} a\right)}{2 \omega_{n}}}}
    $$
    and $\omega_{n}$ is obtained from the solution of
    $$
    \frac{1}{b}-\omega_{n} \tan \left(\omega_{n} a\right)=0 \quad \text { in the range }\left[(n-1) \frac{\pi}{a},\left(n-\frac{1}{2}\right) \frac{\pi}{a},\right]
    $$

*  For $n \geq 2$ and $n=$ even,
    $$
    \lambda_{n}=\frac{2 b}{1+\omega_{n}^{2} b^{2}}, \quad \varphi_{n}(x)=l_{n} \sin \left(\omega_{n} x\right)
    $$
    with
    $$
    l_{n}=\frac{1}{\sqrt{a-\frac{\sin \left(2 \omega_{n} a\right)}{2 \omega_{n}}}}
    $$
    and $\omega_{n}$ being the solution of
    $$
    \frac{1}{b} \tan \left(\omega_{n} a\right)+\omega_{n}=0 \quad \text { in the range }\left[\left(n-\frac{1}{2}\right) \frac{\pi}{a}, n \frac{\pi}{a},\right]
    $$

These equations will be solved numerically.

---

The truncated Karhunen-Loeve expansion of $M$ principal components is
$$\hat{f}_M(x) = \sum_{n=1}^{M} \sqrt{\lambda_n} \varphi_n(x - a) \xi_n$$
where $\xi_n$ are independent standard normal variables since the process is Gaussian (book page 31).

Since $\xi_n$ are uncorrelated (independent actually) and with variance 1 we have that (book 2.21)

$$\textrm{Var}\left[\hat{f}_M(x)\right] = \sum_{n=1}^{M} \lambda_n \varphi_n^2(x - a)$$

Integrating over all $x$, since we have that $\varphi_n$ are orthonormal, we get the *total explained variance* $\sum_{n=1}^{M} \lambda_n$

Since the variance of $f(x)$ is always 1, integrating it over the domain gives us *total variance* 5.

Therefore the *total explained variance ratio* is given by $\frac{1}{5} \sum_{n=1}^{M} \lambda_n$ and a reasonable value for it is 0.99. This is how we will choose our $M$.

---

We calculate the expected ensemble mean and variance. We use the again the fact that $\xi_n$ are uncorrelated with mean 0 and variance 1. We also use the fact that the eigenvalues are all non negative since the covariance function is positive semidefinite (Mercer's theorem).

We have that $\forall x \in [0, 5]$

$$
\begin{align*}
\textrm{E}\left[\hat{f}_M(x)\right] &= \sum_{n=1}^{M} \sqrt{\lambda_n} \varphi_n(x - a) \textrm{E}\left[ \xi_n \right] = 0\\
\textrm{Var}\left[\hat{f}_M(x)\right] &= \sum_{n=1}^{M} \lambda_n \varphi_n^2(x - a) \leq \sum_{n=1}^{\infty} \lambda_n \varphi_n^2(x - a) = \textrm{Var}[f(x)] = 1
\end{align*}
$$


Thus, $\forall x \in [0, 5]$

$$
\begin{align*}
\textrm{E}\left[ \hat{E}_M(x) \right] &= \textrm{E}\left[ 10(1 + \hat{f}_M(x)) \right] = 10  (1 + \textrm{E}\left[ \hat{f}_M(x) \right] ) = 10\\
\textrm{Var}\left[ \hat{E}_M(x) \right] &= 100 \; \textrm{Var}\left[ \hat{f}_M(x) \right] \leq 100
\end{align*}
$$

with the upper bound on the variance being tighter as $M$ increases (equality when $M \to \infty$)


In [1]:
import numpy as np
from scipy.optimize import fsolve, root


a = 2.5
b = 2.0


def eq_odd(omega):
    return 1/b - omega * np.tan(omega * a)

def eq_even(omega):
    return 1/b * np.tan(omega * a) + omega


ratio_thresh = 0.99
total_variance = 2 * a

omegas, lams, cs = [], [], []
explained_variance = 0
n = 1

while explained_variance / total_variance < ratio_thresh:
    if n % 2 == 1:
        interval_center = (n - 0.75) * np.pi / a
        omega = fsolve(eq_odd, interval_center)
        lam = 2*b / (1 + (omega * b)**2)
        c = 1 / np.sqrt(a + np.cos(2*omega*a) / (2*omega))
    else:
        interval_center = (n - 0.25) * np.pi / a
        omega = fsolve(eq_even, interval_center)
        lam = 2*b / (1 + (omega * b)**2)
        c = 1 / np.sqrt(a - np.sin(2*omega*a) / (2*omega))
    
    lams.append(lam.item())
    omegas.append(omega.item())
    cs.append(c.item())
    
    explained_variance += lam
    n += 1
    
    if n == 1_000:
        break
    
    
print(f'Explained Variance Ratio: {explained_variance / total_variance}')


NameError: name 'np' is not defined

In [17]:
import numpy as np
from scipy.optimize import fsolve

def omegas(wtype, a=2.5, b=2.0,p=100):
    ws = np.zeros(p)
    equation = eq_odd if wtype == 'cos' else eq_even
    subt = 1.0 if wtype =='cos' else 0.5
    for idx in range(1,p+1):
        ws[idx -1] = fsolve(equation ,(((idx -subt)*np.pi)/a)+1e-4)[0]
    return ws

def lambdas(ws ,b=2.0):
    return (2.0*b)/(1 + (ws*b)**2)

w_cos = omegas(wtype='cos',p=100)
w_sin = omegas(wtype='sin',p=100)
EV = 0.2*( lambdas(w_cos )+ lambdas(w_sin )).sum()

EV

0.9974606622959471